## 第一部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: https://lab.github.com/githubtraining/introduction-to-github ;
     Jupyter一般做实验测试展示使用，比较方便，可视化效果比较好；
     Pycharm一般在大型工程上使用。

#### 2. What's the Probability Model?

Ans: $ Pro(w_1 w_2 w_3 w_4) = Pr(w_1 | w_2 w_3 w_ 4) * P(w2 | w_3 w_4) * Pr(w_3 | w_4) * Pr(w_4)$ 

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans: 句子生成，语音识别。

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans: 因为使用概率模型可以通过大量数据获取每个词出现的概率，从而可以得到句子的概率，来生成概率最大的句子。

#### 5. What's the Language Model;

Ans: $ language\_model(String) = Probability(String) \in (0, 1) $ 语言模型是获取句子的概率

#### 6. Can you came up with some sceneraies at which we could use Language Model?


Ans: 句子生成，语音识别，分词算法，机器翻译。

#### 7. What's the 1-gram language model;

Ans: 当前词的概率与其他词都无关。$ Pro(w_1 w_2 w_3 w_4) = Pr(w1) * P(w2) * Pr(w_3) * Pr(w_4)$ 

#### 8. What's the disadvantages and advantages of 1-gram language model;

Ans: 1-gram的优点是方便计算，缺点是没有考虑到其他词的影响，算出来一个句子的概率可能会有误差。

#### 9. What't the 2-gram models;

Ans: 当前词的概率与前面一个词有关。$ Pro(w_1 w_2 w_3 w_4) = Pr(w_1 | w_2) * P(w2 | w_3) * Pr(w_3 | w_4) * Pr(w_4)$

## 第二部分

#### 1. 设计你自己的句子生成器

第一个语法：

In [55]:
sentence_1 = '''
sentence => 队伍 球员 动作
队伍 => 火箭队的 | 湖人队的 | 勇士队的
球员 => 哈登 | 詹姆斯 | 库里
动作 => 扣篮 | 投三分 | 抢断 | 盖帽

'''

In [122]:
sentence_2 = '''
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase verber
Article =>  一个 | 这个 |有个
noun =>   男孩 |  女孩 | 桌子 | 小猫
verb =>   看着 |  坐在 |  听着 | 看见
verber =>  写代码 | 打篮球 | 做作业 
Adj =>  蓝色的 | 好看的 | 小小的
'''

In [57]:
def creat_grammar(grammar_str, split_str = '=>'):
    grammar = {}
    for line in grammar_str.split('\n'):
        if not line.strip():
            continue
        exp, stmt = line.split(split_str)
        grammar[exp.strip()] = [i.split() for i in stmt.split('|')]
    return grammar

In [58]:
import random

In [95]:
def generate(gram, target):
    if target not in gram:
        return target
    expand = [generate(gram, t) for t in random.choice(gram[target])]
    return ''.join([e for e in expand if e != 'null'])

In [247]:
def generate_n(n=5):
    n_sentence = []
    for i in range(n):
        n_sentence.append(generate(creat_grammar(sentence_2), 'sentence'))
    return n_sentence

In [124]:
generate_n(10)

['这个女孩听着一个小猫做作业',
 '这个桌子看见这个女孩打篮球',
 '勇士队的库里盖帽',
 '有个小小的好看的小小的小猫看见有个好看的桌子打篮球',
 '有个女孩看见有个好看的好看的桌子打篮球',
 '火箭队的库里扣篮',
 '一个好看的蓝色的女孩看着一个女孩做作业',
 '火箭队的詹姆斯盖帽',
 '火箭队的库里扣篮',
 '有个桌子听着一个蓝色的女孩打篮球']

#### 2. 使用新数据源完成语言模型的训练


In [220]:
import pandas as pd
import jieba
import re
from collections import Counter
import jieba
from functools import reduce
from operator import add, mul
from collections import Counter
import numpy as np

In [131]:
data = pd.read_csv('./movie_comments.csv')

/Users/liujunhong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [129]:
len(data)

261497

In [132]:
data.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [133]:
comment = data['comment'].tolist()

In [137]:
def token(string):
    return re.findall('\w+', string)

In [154]:
comment_clean = []

In [155]:
for i in comment:
    if type(i) != str: continue
    comment_clean.append(''.join(token(i)))

In [157]:
len(comment_clean)

261495

In [176]:
def cut(string): return list(jieba.cut(string))

In [177]:
comment_cut = []

In [178]:
for i in comment_clean:
    comment_cut += cut(i)

In [180]:
len(comment_cut)

4490311

In [182]:
words_count = Counter(comment_cut)

In [196]:
comment_cut_2_GRAM = [''.join(comment_cut[i:i+2]) for i in range(len(comment_cut[:-2]))]

In [198]:
len(comment_cut_2_GRAM)

4490309

In [199]:
words_count_2 = Counter(comment_cut_2_GRAM)

In [201]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / len(comment_cut_2_GRAM)
    else:
        return 1 / len(comment_cut_2_GRAM)

In [203]:
prob_2('我','和')

4.4985768240003084e-05

In [206]:
def get_prob(sentence):
    words = cut(sentence)
    sentence_prob = 1
    
    for i,word in enumerate(words[:-1]):
        sentence_prob *= prob_2(words[i],words[i+1])
    
    return sentence_prob

In [208]:
get_prob('我要去吃饭')

2.083036282080613e-12

#### 3. 获得最优质的的语言

In [258]:
def generate_best(n=5): 
    score = []
    sentences = generate_n(n)
    for i in sentences:
        score.append(get_prob(i))
    return sentences[np.array(score).argmax()]

In [264]:
generate_best(100)

'一个男孩看见这个女孩做作业'

Q: 这个模型有什么问题？ 你准备如何提升？ 

Ans: 数据的场景可能比较局限，算出来的概率可能不准确，需要通过扩充数据集，结合多个不同场景的数据，来提升模型的准确度。